<a href="https://colab.research.google.com/github/timlamec/work/blob/master/Image_collection_and_Supervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Installs geemap package
import subprocess
 
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
 
# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap
 
# Authenticates and initializes Earth Engine
import ee
 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=oJUoLGgAuKK0ozO2-dbeKmm_2eiLVvr99CSFK0HG5H8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi1Ujo3myxE7w6xxF75C_DMfaa3UQOw_k1zZ-JRePH1cSvoO_fm8Nk

Successfully saved authorization token.


In [ ]:
# !pip install geopandas 
# !pip -q install rasterio

# 1 Import Libraries

In [ ]:
import ee
import geemap
import geemap.eefolium
#import rasterio
#from rasterio import plot
#%matplotlib inline

# 2 Define Area of interest

Here replace the counties affected with your google earth engine asset or use a shapefile and convert it to GEE asset using the function in section *3* you may also subset the various counties. 

In [ ]:
# Counties Affected Area of interest
Counties_affected =  ee.FeatureCollection('users/collinsasegaca/Affected_counties') #https://drive.google.com/file/d/1VAZ879vzOUv9Hbo2PWFwCczYIGWityfS/view?usp=sharing
naiserian = ee.FeatureCollection('users/collinsasegaca/naiserian') 
# # Subset the counties since the entire region takes too much time to compute  #link to the shapefiles https://drive.google.com/drive/folders/1eFN6ZZKN3f-KHJMx4AaDP2Ko3psUOb6W?usp=sharing
# Elgeiyo_Baringo = './drive/MyDrive/omdena/affected_counties/brokendown/Elgeyo_baringo.shp' # Elgeiyo Marakwet & Baringo 
# Laikipia_samburu = './drive/MyDrive/omdena/affected_counties/brokendown/Laikipia_samburu.shp' #  Laikipia_samburu
# Kitui = './drive/MyDrive/omdena/affected_counties/brokendown/Kitui.shp' # Kitui
# Mandera = './drive/MyDrive/omdena/affected_counties/brokendown/Mandera.shp' # Mandera
# Machakos_embu = './drive/MyDrive/omdena/affected_counties/brokendown/Machakos_embu.shp' # Machakos_embu
# Marsabit = './drive/MyDrive/omdena/affected_counties/brokendown/Marsabit.shp' # Marsabit
# Meru_isolo ='./drive/MyDrive/omdena/affected_counties/brokendown/Meru_isolo.shp' # Meru and Isiolo
# Tana = './drive/MyDrive/omdena/affected_counties/brokendown/Tana.shp' #Tana
# Turkana = './drive/MyDrive/omdena/affected_counties/brokendown/Turkana.shp' 
# Wajir = './drive/MyDrive/omdena/affected_counties/brokendown/Wajir.shp' #Wajir 
# Garissa = './drive/MyDrive/omdena/affected_counties/brokendown/Garissa.shp' # Garisa

Rwanda = ee.FeatureCollection("users/lericnganga/RWANDA/Rwanda_Boundary_1kmbuffer");
Rwanda_TrainingData_2021_v5 = ee.FeatureCollection("users/lericnganga/RWANDA/Rwanda_TrainingData_2021_v5")

 
# 

In [ ]:
def maskS2clouds(image): 
   qa = image.select('QA60');

   # Bits 10 and 11 are clouds and cirrus, respectively.
   cloudBitMask = 1 << 10;
   cirrusBitMask = 1 << 11;

   # Both flags should be set to zero, indicating clear conditions.
   mask = qa.bitwiseAnd(cloudBitMask).eq(0)#and(qa.bitwiseAnd(cirrusBitMask).eq(0));

   return image.updateMask(mask).divide(10000);

3 Convert shapefiles to google earth engine objects 

In [ ]:
def conv(in_shp):
  ee_object = geemap.shp_to_ee(in_shp)
  return ee_object

4 Define the dates

In [ ]:
# date before attack
fdate = '2019-08-1' #('2017-01-1' ,'2017-12-31')
ldate = '2019-12-25'
 
#date after attack
after_fdate = '2020-1-1'
after_ldate = '2020-5-31'

# 5 Image collection

In [ ]:
# NDVI function for L8
def NDVI_L8(image):
  ndvi= image.normalizedDifference(['B5','B4']).rename('NDVI') #Compute the NDVI values while Using the NIR and Red Bands
  

  return image.addBands(ndvi)

In [ ]:
def sentinel_2_before(geometry):
  collection = ee.ImageCollection("COPERNICUS/S2") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .filterBounds(geometry) \
 
  before_collection = collection.filterDate(fdate,ldate)# before attack
  
  medianpixels_before = before_collection.map(maskS2clouds).median()
  medianpixelsclipped_before = medianpixels_before.clip(geometry) #.divide(10000)
  return medianpixelsclipped_before

In [ ]:

Rwanda_image = sentinel_2_before(Rwanda)

In [ ]:

vis_params = {
    'min': 0,
    'max': 1,
    'bands': ['B12', 'B11', 'B4'], 'gamma': 1.5
  }

6 Visualize the Image

In [ ]:
NDVI = NDVI_L8(All_Counties_after)

In [ ]:
ndvi_palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301'];
vis_ndvi ={'min':0, 'max':3,'palette':ndvi_palette}

In [ ]:
# NDVI.bandNames().getInfo()

In [ ]:
Map_p = geemap.Map()
Map_p.addLayer(Rwanda_image, vis_params, 'Rwanda',1)
Map_p.addLayer(classify_Rwanda,vis_ndvi, 'NDVI')
Map_p.centerObject(Rwanda, 8)
Map_p.addLayerControl()
Map_p

Map(center=[-2.0014515352750792, 29.925550450611976], controls=(WidgetControl(options=['position', 'transparen…

7 Build a webmap visualization 


In [ ]:
Map_p.save('counties_affected_before&after.html',close_file=True)

Getting images for respective counties 
here you only read to read a shapefile from your drive and use the function to convert to gee object 

Generate training points from sentinel 2 2016 LULC image for kenya 

link= http://geoportal.rcmrd.org/layers/servir%3Akenya_sentinel2_lulc2016

8 Train the model and classify the images

In [ ]:
# Training and classifiy
def Train_classify(Image,points): 
  bands = ['B2', 'B3', 'B4', 'B8']
 
 
# This property of the table stores the land cover labels.
  label = 'LC_Code'
 
# Overlay the points on the imagery to get training for the before imge
  training = Image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 10
  })
 
# Train a CART classifier with default parameters.
  trained = ee.Classifier.smileCart().train(training, label, bands)
  # Classify the image with the same bands used for training.
  result = image.select(bands).classify(trained)
  return result

In [ ]:
classify_Rwanda.bandNames().getInfo()

['classification']

In [ ]:
classify_Rwanda = Train_classify(Rwanda_image,Rwanda_TrainingData_2021_v5)

In [ ]:
#Export result
geemap.ee_export_image_to_drive(Garrisa_Classified_before, description='Garissa_before', folder='omdena', scale=10)

Exporting Garissa_before ...


## 9 Accuracy assement 
## Overall Accuracy essentially tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference site were classified correctly

There are various methods to check accuracy including:

***Kappa***
The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random.

***confusionMatrix()***

# confusionMatrix() computes a 2D confusion matrix for a classifier based on its training data (ie: resubstitution error). Axis 0 of the matrix correspond to the input classes (i.e., reference data), and axis 1 to the output classes (i.e., classification data). The rows and columns start at class 0 and increase sequentially up to the maximum class value





In [ ]:
#Split Training and testing
def train_test(train_Test):
  bands = ['B2', 'B3', 'B4', 'B8']


  # This property of the table stores the land cover labels.
  label = 'LC_Code'

  # Overlay the points on the imagery to get training for the before imge
  sample = image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 10
  })

  # Adds a column of deterministic pseudorandom numbers. 
  sample = sample.randomColumn()

  split = 0.7

  training_dta = sample.filter(ee.Filter.lt('random', split))
  validation_dta = sample.filter(ee.Filter.gte('random', split))
  #inspect the data
  training_dta.first().getInfo()
  validation_dta.first().getInfo()
  # Train Classifier
  classifier = ee.Classifier.smileRandomForest(10).train(training_dta, label, bands)
  # Classify the image with the same bands used for training.
  result_1 = image.select(bands).classify(classifier)
  train_accuracy = classifier.confusionMatrix()
  train_accuracy.getInfo()
  train_accuracy.accuracy().getInfo()
  print('confusionMatrix:',train_accuracy.accuracy().getInfo())
  print('Kappa:',train_accuracy.kappa().getInfo())
  validated = validation_dta.classify(classifier) # use the validation dataset
  #test accuracy
  test_accuracy = validated.errorMatrix('LC_Code', 'classification') # errorMatrix computes a 2D error matrix for a collection by comparing two columns of a collection: one containing the actual values, and one containing predicted values.
  test_accuracy.accuracy().getInfo()
  return result_1

In [ ]:
image = Rwanda_image
points = Rwanda_TrainingData_2021_v5

In [ ]:
accuracy_Garisaa = train_test(image)

confusionMatrix: 0.9564053537284894
Kappa: 0.9344999759097858


In [ ]:
accuracy_Garisaa.bandNames().getInfo()

['classification']

Elgeiyo Markwet and Baringo county

Accuracy asessment 

In [ ]:
image = Elgyo_Baringo_image
points = points_Elgeiyo_baringo

NameError: ignored

In [ ]:
accuracy_Elgyo_Baringo = train_test(image)


confusionMatrix: 0.928494430470715
Kappa: 0.9012755717465486


In [ ]:
#Export Before and After Images for 
# geemap.ee_export_image_to_drive(Elgyo_Baringo_Classified_After, description='Elgeiyo_Baringo_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Elgyo_Baringo_Classified_before, descripti='Elgeiyo_Baringo_before', folder='omdena', scale=10)

Kitui

In [ ]:
AOI = conv(Kitui) # define AOI for the county
Kitui_image_before = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Kitui = training(ROI) #this generates points from the function


# Training and classifiy

image = Kitui_image_before # defines the image for the Location
points = points_Kitui # defines the points to be used 
Kitui_Classified_before = Train_classify(image)



In [ ]:
#after 
#AOI = conv(E) # define AOI for the county
Kitui_after = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Kitui = training(ROI) #this generates points from the function

# Training and classifiy

image = Kitui_after # defines the image for the Location
points = points_Kitui # defines the points to be used 
Kitui_Classified_After = Train_classify(image) 

Accuracy assesment 

In [ ]:
#Before image
image = Kitui_image_before
accuracy_Kitui = train_test(image)

confusionMatrix: 0.9268590004063388
Kappa: 0.8948542375215397


In [ ]:
# After image
image = Kitui_after
accuracy_Kitui = train_test(image)

confusionMatrix: 0.9405619596541787
Kappa: 0.8844543009598962


In [ ]:
# Export Images 
# geemap.ee_export_image_to_drive(Kitui_Classified_After, description='Kitui_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Kitui_Classified_before, description='Kitui_Classified_before', folder='omdena', scale=10)

Laikipia and Samburu

In [ ]:
AOI = conv(Laikipia_samburu) # define AOI for the county
Laikipia_Samburu_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Laikipia_Samburu = training(ROI) #this generates points from the function

# Training and classifiy

image = Laikipia_Samburu_before_image # defines the image for the Location
points = points_Laikipia_Samburu # defines the points to be used 
Laikipia_Samburu_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Laikipia_Samburu_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Laikipia_Samburu_after_image # defines the image for the Location
points = points_Laikipia_Samburu # defines the points to be used 
Laikipia_Samburu_Classified_after = Train_classify(image)



Accuracy

In [ ]:
#Before image
image = Laikipia_Samburu_before_image
accuracy_Kitui = train_test(image)

confusionMatrix: 0.9258064516129032
Kappa: 0.8848680574212912


In [ ]:
#Atfer image
image = Laikipia_Samburu_after_image
accuracy_Laikipia_samburu = train_test(image)

confusionMatrix: 0.9245216158752658
Kappa: 0.8893573647401369


In [ ]:
#export the images
# geemap.ee_export_image_to_drive(Laikipia_Samburu_Classified_after, description='Laikipia_samburu_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Laikipia_Samburu_Classified_before, description='Laikipia_samburu_Classified_before', folder='omdena', scale=10)

Machakos Embu

In [ ]:
AOI = conv(Machakos_embu) # define AOI for the county
Machakos_embu_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Machakos_embu = training(ROI) #this generates points from the function

# Training and classifiy

image = Machakos_embu_before_image # defines the image for the Location
points = points_Machakos_embu # defines the points to be used 
Machakos_embu_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Machakos_embu_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Machakos_embu_after_image # defines the image for the Location
points = points_Machakos_embu # defines the points to be used 
Machakos_embu_Classified_after = Train_classify(image)

Accuracy Assesment

In [ ]:
#Before image
image = Machakos_embu_before_image
accuracy_Machakos_embu = train_test(image)

confusionMatrix: 0.9563577586206896
Kappa: 0.8908268360625405


In [ ]:
# After image 
image = Machakos_embu_after_image
accuracy_Machakos_embu = train_test(image)

confusionMatrix: 0.9593184979137691
Kappa: 0.8896768437011812


In [ ]:
# geemap.ee_export_image_to_drive(Machakos_embu_Classified_after, description='Machakos_Embu_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Machakos_embu_Classified_before, description='Machakos_Embu_Classified_before', folder='omdena', scale=10)

Mandera 

In [ ]:
AOI = conv(Mandera) # define AOI for the county
Mandera_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Mandera = training(ROI) #this generates points from the function

# Training and classifiy

image = Mandera_before_image # defines the image for the Location
points = points_Mandera # defines the points to be used 
Mandera_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Mandera_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Mandera_after_image # defines the image for the Location
points = points_Mandera # defines the points to be used 
Mandera_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Mandera_before_image
accuracy_Kitui = train_test(image)


confusionMatrix: 0.9318840579710145
Kappa: 0.8669257172529383


In [ ]:
# After Image
image = Mandera_after_image
accuracy_Mandera = train_test(image)

confusionMatrix: 0.9365645046329294
Kappa: 0.8716484678654092


In [ ]:
#Export Images 
# geemap.ee_export_image_to_drive(Mandera_Classified_after, description='Mandera_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Mandera_Classified_before, description='Mandera_Classified_before', folder='omdena', scale=10)

Marsabi

In [ ]:
AOI = conv(Marsabit) # define AOI for the county
Marsabit_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Marsabit = training(ROI) #this generates points from the function

# Training and classifiy

image = Marsabit_before_image # defines the image for the Location
points = points_Marsabit # defines the points to be used 
Marsabit_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Marsabit_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Marsabit_after_image # defines the image for the Location
points = points_Marsabit # defines the points to be used 
Marsabit_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Marsabit_before_image
accuracy_Marsabit = train_test(image)

confusionMatrix: 0.9281183932346723
Kappa: 0.8859419467708444


In [ ]:
# After Image
image = Marsabit_after_image
accuracy_Marsabit = train_test(image)

confusionMatrix: 0.9301152737752162
Kappa: 0.8902403755516359


In [ ]:
#Export Images 
# geemap.ee_export_image_to_drive(Marsabit_Classified_after, description='Marsabit_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Marsabit_Classified_before, description='Marsabit_Classified_before', folder='omdena', scale=10)

Meru Isiolo

In [ ]:
AOI = conv(Meru_isolo) # define AOI for the county
Meru_isolo_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Meru_isolo = training(ROI) #this generates points from the function

# Training and classifiy

image = Meru_isolo_before_image # defines the image for the Location
points = points_Meru_isolo # defines the points to be used 
Meru_isolo_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Meru_isolo_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Meru_isolo_after_image # defines the image for the Location
points = points_Meru_isolo # defines the points to be used 
Meru_isolo_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Meru_isolo_before_image
accuracy_Meru_isiolo = train_test(image)

confusionMatrix: 0.9342152306042188
Kappa: 0.9016905320919149


In [ ]:
# After Image
image = Meru_isolo_after_image
accuracy_Meru_isiolo = train_test(image)

confusionMatrix: 0.9247159090909091
Kappa: 0.8861304234212065


In [ ]:
# geemap.ee_export_image_to_drive(Meru_isolo_Classified_after, description='Meru_isiolo_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Meru_isolo_Classified_before, description='Meru_isiolo_Classified_before', folder='omdena', scale=10)

Tana

In [ ]:
AOI = conv(Tana) # define AOI for the county
Tana_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Tana = training(ROI) #this generates points from the function

# Training and classifiy

image = Tana_before_image # defines the image for the Location
points = points_Tana # defines the points to be used 
Tana_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Tana_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Tana_after_image # defines the image for the Location
points = points_Tana # defines the points to be used 
Tana_Classified_after = Train_classify(image)

accuracy tana

In [ ]:
# Before Image
image = Tana_before_image
accuracy_Tana = train_test(image)

confusionMatrix: 0.9228430254656025
Kappa: 0.8476276416951698


In [ ]:
# After Image
image = Tana_after_image
accuracy_Tana = train_test(image)

confusionMatrix: 0.9276504297994269
Kappa: 0.8637948789285665


In [ ]:
AOI = conv(Turkana) # define AOI for the county
Turkana_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Turkana = training(ROI) #this generates points from the function

# Training and classifiy

image = Turkana_before_image # defines the image for the Location
points = points_Turkana # defines the points to be used 
Turkana_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Turkana_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Turkana_after_image # defines the image for the Location
points = points_Turkana # defines the points to be used 
Turkana_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Turkana_before_image
accuracy_Turkana = train_test(image)

confusionMatrix: 0.9261818181818182
Kappa: 0.881316847403398


In [ ]:
# After Image
image = Turkana_after_image
accuracy_Turkana = train_test(image)

confusionMatrix: 0.9295470884255931
Kappa: 0.8934621726353122


In [ ]:
# Export Images 
# geemap.ee_export_image_to_drive(Tana_Classified_after, description='Tana_Classified_after', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Tana_Classified_before, description='Tana_Classified_before', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Turkana_Classified_after, description='Turkana_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Turkana_Classified_before, description='Turkana_Classified_before', folder='omdena', scale=10)

Wajir

In [ ]:
AOI = conv(Wajir) # define AOI for the county
Wajir_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Wajir = training(ROI) #this generates points from the function

# Training and classifiy

image = Wajir_before_image # defines the image for the Location
points = points_Wajir # defines the points to be used 
Wajir_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Wajir_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Wajir_after_image # defines the image for the Location
points = points_Wajir # defines the points to be used 
Wajir_Classified_after = Train_classify(image)

accuracy Wajir

In [ ]:
# Before image
image = Wajir_before_image
accuracy_Wajir = train_test(image)

confusionMatrix: 0.9335740072202167
Kappa: 0.8757139958798927


In [ ]:
#After Image
image = Wajir_after_image
accuracy_Wajir = train_test(image)

confusionMatrix: 0.9252669039145908
Kappa: 0.860839715934737


Tharaka Nithi

In [ ]:
tharaka = './drive/MyDrive/omdena/tharaka_shapefile/tharaka_nithi.shp'

In [ ]:

AOI = conv(tharaka) # define AOI for the county
tharaka_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_tharaka = training(ROI) #this generates points from the function

# Training and classifiy

image = tharaka_before_image # defines the image for the Location
points = points_tharaka # defines the points to be used 
tharaka_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
tharaka_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = tharaka_after_image # defines the image for the Location
points = points_tharaka # defines the points to be used 
tharaka_Classified_after = Train_classify(image)

accuracy Tharaka

In [ ]:
# Before image
image = tharaka_before_image
accuracy_Tharaka = train_test(image)

confusionMatrix: 0.9658273381294964
Kappa: 0.9226254041218598


In [ ]:
#After Image
image = tharaka_after_image
accuracy_Tharaka = train_test(image)

confusionMatrix: 0.9609598853868195
Kappa: 0.9094364058339849


In [ ]:
#Export tharaka
# geemap.ee_export_image_to_drive(tharaka_Classified_after, description='Tharaka_Classified_after', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(tharaka_Classified_before, description='Tharaka_Classified_before', folder='omdena', scale=10)

In [ ]:
#export Wajir
# geemap.ee_export_image_to_drive(Wajir_after_image, description='Wajir_Classified_after', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Wajir_Classified_before, description='Wajir_Classified_before', folder='omdena', scale=10)
